In [1]:
import pandas as pd
import re

In [2]:
#annotations=pd.read_csv("genomad_annotations.tsv", sep = "\t")
annotations=pd.read_csv("genomad_annotations.tsv", sep = "\t")

In [3]:
annotations.head()

,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,ko_id,kegg_hit,...,peptidase_RBH,peptidase_identity,peptidase_bitScore,peptidase_eVal,pfam_hits,cazy_ids,cazy_hits,cazy_subfam_ec,cazy_best_hit,heme_regulatory_motif_count
0,NASQAN2010_121_B_1000_plasmid___k121_111206_1,NASQAN2010_121_B_1000_plasmid__,k121_111206,1,1,522,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,Binding-protein-dependent transport system inn...,NaN,NaN,NaN,NaN,0
1,NASQAN2010_121_B_1000_plasmid___k121_111206_2,NASQAN2010_121_B_1000_plasmid__,k121_111206,2,519,1400,1,C,K17317,glucose/mannose transport system permease protein,...,NaN,NaN,NaN,NaN,Binding-protein-dependent transport system inn...,NaN,NaN,NaN,NaN,0
2,NASQAN2010_121_B_1000_plasmid___k121_111206_3,NASQAN2010_121_B_1000_plasmid__,k121_111206,3,1404,2516,1,C,K10112,multiple sugar transport system ATP-binding pr...,...,NaN,NaN,NaN,NaN,ABC transporter [PF00005.30]; MalK OB fold dom...,NaN,NaN,NaN,NaN,0
3,NASQAN2010_121_B_1000_plasmid___k121_111206_4,NASQAN2010_121_B_1000_plasmid__,k121_111206,4,2513,3157,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,GMC oxidoreductase [PF00732.22]; FAD binding d...,NaN,NaN,NaN,NaN,0
4,NASQAN2010_121_B_1000_plasmid___k121_113216_1,NASQAN2010_121_B_1000_plasmid__,k121_113216,1,2,448,-1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,Chromate resistance exported protein [PF09828....,NaN,NaN,NaN,NaN,0


In [9]:
pfam2gofile="pfam2go.txt"
pfam2godict={}

with open(pfam2gofile, 'r') as file:
    for line in file:
        line=line.strip()
        pattern='Pfam:(PF\d*)\s.*;\s(GO:.*)'
        groups = re.search(pattern, line)
        if groups:
            pfam=str(groups.group(1))
            go=str(groups.group(2))
            if pfam not in pfam2godict:
                pfam2godict[pfam] = [go]
            else:
                pfam2godict[pfam].append(go)
            #pfam2godict[str(groups.group(1))] = str(groups.group(2))
            #print(type(groups.group(1)))
        #else:
            #print(line)

In [40]:
def pfamtogocounts(df):
    pattern=r'\[(.*?)\.\d*?\]'
    godf = pd.DataFrame()
    for index, annotation in df.iterrows():
        if not pd.isna(annotation['pfam_hits']):
            #print(annotation['pfam_hits'])
            pfams=re.findall(pattern, annotation['pfam_hits'])
            for pfam in pfams:
                if pfam in pfam2godict:
                    for value in pfam2godict[pfam]:
                        #gorow=pd.DataFrame({'GO': value, 'contig': [annotation['Unnamed: 0']]})
                        gorow=pd.DataFrame({'GO': value, 'contig': [annotation['Unnamed: 0.1']]})
                        godf=godf.append(gorow)
    godf = godf.drop_duplicates(subset=['GO', 'contig'], keep="first")
    gocounts=pd.DataFrame(godf['GO'].value_counts())
    return(gocounts)


In [14]:
godf.head()

,GO,contig
0,GO:0055085,NASQAN2010_121_B_1000_plasmid___k121_111206_1
0,GO:0016020,NASQAN2010_121_B_1000_plasmid___k121_111206_1
0,GO:0055085,NASQAN2010_121_B_1000_plasmid___k121_111206_2
0,GO:0016020,NASQAN2010_121_B_1000_plasmid___k121_111206_2
0,GO:0005524,NASQAN2010_121_B_1000_plasmid___k121_111206_3


In [17]:
godf = godf.drop_duplicates(subset=['GO', 'contig'], keep="first")

In [19]:
gocounts=pd.DataFrame(godf['GO'].value_counts())

In [20]:
gocounts.head()

,GO
GO:0003677,16076
GO:0016020,11956
GO:0005524,11004
GO:0006355,10243
GO:0055085,9299


In [21]:
gocounts.to_csv('defaultcounts.csv')

In [74]:
godf.to_csv('gos.tsv', sep="\t", index=False, header=False)

In [24]:
conservativegos = pd.read_csv('conservative_annotations.tsv', sep ="\t")
conservativegos.head()

,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,ko_id,kegg_hit,...,peptidase_RBH,peptidase_bitScore,peptidase_eVal,peptidase_identity,pfam_hits,cazy_ids,cazy_hits,cazy_subfam_ec,cazy_best_hit,heme_regulatory_motif_count
0,NASQAN2010_127_B_1000_plasmid_conservative___k...,NASQAN2010_127_B_1000_plasmid_conservative__,k121_43221,1,1,1017,-1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NASQAN2010_127_B_1000_plasmid_conservative___k...,NASQAN2010_127_B_1000_plasmid_conservative__,k121_606620,1,1,1086,-1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,MobA/MobL family [PF03389.18],NaN,NaN,NaN,NaN,0
2,NASQAN2010_127_B_1000_plasmid_conservative___k...,NASQAN2010_127_B_1000_plasmid_conservative__,k121_634694,1,2,466,1,C,K08309,soluble lytic murein transglycosylase [EC:4.2....,...,False,85.0,4.429000e-18,0.618,Transglycosylase SLT domain [PF01464.23],GH23,GH23 lysozyme type G (EC 3.2.1.17); peptidogly...,"3.2.1.17,3.2.1.17_or_4.2.2.n1,4.2.2.n1",GH23.hmm,0
3,NASQAN2010_127_B_1000_plasmid_conservative___k...,NASQAN2010_127_B_1000_plasmid_conservative__,k121_634694,2,475,792,1,C,K03197,type IV secretion system protein VirB2,...,NaN,NaN,NaN,NaN,TrbC/VIRB2 pilin [PF04956.16],NaN,NaN,NaN,NaN,0
4,NASQAN2010_127_B_1000_plasmid_conservative___k...,NASQAN2010_127_B_1000_plasmid_conservative__,k121_634694,3,796,1080,1,C,K03198,type IV secretion system protein VirB3,...,NaN,NaN,NaN,NaN,"Type IV secretory pathway, VirB3-like protein ...",NaN,NaN,NaN,NaN,0


In [25]:
conservativecounts=pfamtogocounts(conservativegos)

In [26]:
conservativecounts.head()

,GO
GO:0003677,1046
GO:0016020,787
GO:0005524,567
GO:0006355,306
GO:0003887,299


In [27]:
conservativecounts.to_csv('conservativecounts.csv')

In [28]:
calibrated = pd.read_csv('calibratedgenomadannotations.tsv', sep = "\t")
calibrated.head()

,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,ko_id,kegg_hit,...,peptidase_RBH,peptidase_identity,peptidase_bitScore,peptidase_eVal,pfam_hits,heme_regulatory_motif_count,cazy_ids,cazy_hits,cazy_subfam_ec,cazy_best_hit
0,NASQAN2010_121_B_1000_plasmid_calibrated___k12...,NASQAN2010_121_B_1000_plasmid_calibrated__,k121_111206,1,1,522,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,Binding-protein-dependent transport system inn...,0,NaN,NaN,NaN,NaN
1,NASQAN2010_121_B_1000_plasmid_calibrated___k12...,NASQAN2010_121_B_1000_plasmid_calibrated__,k121_111206,2,519,1400,1,C,K17317,glucose/mannose transport system permease protein,...,NaN,NaN,NaN,NaN,Binding-protein-dependent transport system inn...,0,NaN,NaN,NaN,NaN
2,NASQAN2010_121_B_1000_plasmid_calibrated___k12...,NASQAN2010_121_B_1000_plasmid_calibrated__,k121_111206,3,1404,2516,1,C,K10112,multiple sugar transport system ATP-binding pr...,...,NaN,NaN,NaN,NaN,ABC transporter [PF00005.30]; MalK OB fold dom...,0,NaN,NaN,NaN,NaN
3,NASQAN2010_121_B_1000_plasmid_calibrated___k12...,NASQAN2010_121_B_1000_plasmid_calibrated__,k121_111206,4,2513,3157,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,GMC oxidoreductase [PF00732.22]; FAD binding d...,0,NaN,NaN,NaN,NaN
4,NASQAN2010_121_B_1000_plasmid_calibrated___k12...,NASQAN2010_121_B_1000_plasmid_calibrated__,k121_126471,1,1,393,-1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


In [29]:
calibratedcounts=pfamtogocounts(calibrated)
calibratedcounts.head()

,GO
GO:0003677,12155
GO:0016020,6828
GO:0005524,6601
GO:0006355,6026
GO:0016491,4968


In [31]:
calibratedcounts.to_csv('calibratedcounts.csv')

In [33]:
unfiltered = pd.read_csv('GROWunfilteredplatonannotations.tsv', sep = "\t")
unfiltered.head()

,Unnamed: 0,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,ko_id,kegg_hit,...,peptidase_RBH,peptidase_identity,peptidase_bitScore,peptidase_eVal,pfam_hits,cazy_ids,cazy_hits,cazy_subfam_ec,cazy_best_hit,heme_regulatory_motif_count
0,NASQAN2010_127_B_1000.plasmid___k121_108115_1,NASQAN2010_127_B_1000.plasmid__,k121_108115,1,128,1477,1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NASQAN2010_127_B_1000.plasmid___k121_108115_2,NASQAN2010_127_B_1000.plasmid__,k121_108115,2,1637,5449,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,Reverse transcriptase (RNA-dependent DNA polym...,NaN,NaN,NaN,NaN,0
2,NASQAN2010_127_B_1000.plasmid___k121_108115_3,NASQAN2010_127_B_1000.plasmid__,k121_108115,3,5553,5747,-1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,NASQAN2010_127_B_1000.plasmid___k121_180373_1,NASQAN2010_127_B_1000.plasmid__,k121_180373,1,2,2800,1,D,NaN,NaN,...,NaN,NaN,NaN,NaN,Reverse transcriptase (RNA-dependent DNA polym...,NaN,NaN,NaN,NaN,0
4,NASQAN2010_127_B_1000.plasmid___k121_180373_2,NASQAN2010_127_B_1000.plasmid__,k121_180373,2,3124,6504,1,E,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [34]:
unfilteredcounts=pfamtogocounts(unfiltered)
unfilteredcounts.head()

,GO
GO:0003677,3325
GO:0005524,1614
GO:0006310,1583
GO:0015074,1301
GO:0006260,1183


In [35]:
unfilteredcounts.to_csv("unfilteredcounts.csv")

In [36]:
filtered=pd.read_csv("GROWcanonfilteredplatonannotation.csv")
filtered.head()

,Unnamed: 0,Unnamed: 0.1,fasta,scaffold,gene_position,start_position,end_position,strandedness,rank,ko_id,...,peptidase_RBH,peptidase_identity,peptidase_bitScore,peptidase_eVal,pfam_hits,cazy_ids,cazy_hits,cazy_subfam_ec,cazy_best_hit,heme_regulatory_motif_count
0,14746,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,k121_1081908,1,95,469,-1,E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,14747,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,k121_1081908,2,551,1294,-1,D,NaN,...,NaN,NaN,NaN,NaN,IstB-like ATP binding protein [PF01695.20]; Ba...,NaN,NaN,NaN,NaN,0
2,14748,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,k121_1081908,3,1309,2844,-1,E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,15061,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,k121_1310888,1,522,3437,1,D,NaN,...,NaN,NaN,NaN,NaN,Tn3 transposase DDE domain [PF01526.20]; Domai...,NaN,NaN,NaN,NaN,0
4,15062,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,altamaha_2019_sw_WHONDRS-S19S_0010_B_1000.plas...,k121_1310888,2,3531,3701,1,E,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [41]:
#change the column name in the function
filteredcounts=pfamtogocounts(filtered)
filteredcounts.head()

,GO
GO:0003677,1133
GO:0006310,597
GO:0016020,565
GO:0005524,429
GO:0006355,357


In [42]:
filteredcounts.to_csv("filteredcounts.csv")

In [30]:
def mappfamtogocounts(df):
    df["GO"] = df["pfam_hits"].map(pfam2godict)
    df=df.explode("GO")
    df=df[['GO', 'Unnamed:0']]
    df=df.drop_duplicates()
    dfcounts=pd.DataFrame(df.value_counts())
    return dfcounts
    

In [32]:
mappfamtogocounts(calibrated)

KeyError: "['Unnamed:0'] not in index"

In [23]:
godf.to_csv('defaultgos.csv', index=False, header=False)

In [57]:
pfam2godict.keys()

dict_keys(['PF00001', 'PF00002', 'PF00003', 'PF00004', 'PF00005', 'PF00006', 'PF00009', 'PF00010', 'PF00012', 'PF00013', 'PF00014', 'PF00015', 'PF00016', 'PF00018', 'PF00019', 'PF00020', 'PF00023', 'PF00025', 'PF00028', 'PF00031', 'PF00032', 'PF00033', 'PF00034', 'PF00036', 'PF00039', 'PF00041', 'PF00042', 'PF00044', 'PF00046', 'PF00048', 'PF00049', 'PF00050', 'PF00056', 'PF00057', 'PF00060', 'PF00063', 'PF00064', 'PF00067', 'PF00068', 'PF00069', 'PF00071', 'PF00072', 'PF00073', 'PF00075', 'PF00076', 'PF00080', 'PF00081', 'PF00082', 'PF00083', 'PF00089', 'PF00091', 'PF00093', 'PF00095', 'PF00097', 'PF00098', 'PF00102', 'PF00103', 'PF00105', 'PF00108', 'PF00110', 'PF00111', 'PF00112', 'PF00114', 'PF00115', 'PF00116', 'PF00118', 'PF00119', 'PF00120', 'PF00121', 'PF00123', 'PF00124', 'PF00125', 'PF00126', 'PF00127', 'PF00128', 'PF00131', 'PF00133', 'PF00136', 'PF00137', 'PF00139', 'PF00140', 'PF00141', 'PF00142', 'PF00143', 'PF00145', 'PF00146', 'PF00148', 'PF00149', 'PF00150', 'PF00151',

In [42]:
for annotation in annotations:
    print(annotation)

Unnamed: 0
fasta
scaffold
gene_position
start_position
end_position
strandedness
rank
ko_id
kegg_hit
peptidase_id
peptidase_family
peptidase_hit
peptidase_RBH
peptidase_identity
peptidase_bitScore
peptidase_eVal
pfam_hits
cazy_ids
cazy_hits
cazy_subfam_ec
cazy_best_hit
heme_regulatory_motif_count


In [32]:
re.findall(r'\[(.*?)\]','Transposase [PF01527.23]; Brinker DNA-binding domain [PF09607.13]')

['PF01527.23', 'PF09607.13']